In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sys
import time

In [2]:
import tensorflow as tf
import tensorflow.contrib.keras as keras

from tensorflow.contrib.keras import backend as K
from tensorflow.contrib.keras.python.keras.models import Sequential
from tensorflow.contrib.keras.python.keras.layers import Dense, Dropout, Flatten

In [7]:
print("TensorFlow version =", tf.__version__)
#print("TF contrib Keras version =",keras.__version__)
print("Python version =",sys.version)
print("Keras backend =", keras.backend.backend())

TensorFlow version = 1.3.0
Python version = 3.5.2 (default, Nov 17 2016, 17:05:23) 
[GCC 5.4.0 20160609]
Keras backend = tensorflow


The Unreasonable effectiveness of RNN: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

Minimal char-rnn code: https://gist.github.com/karpathy/d4dee566867f8291f086

In [41]:
def load_data(filename):
    data = open(filename, 'r').read() # should be simple plain text file
    print("Loaded data from", filename)
    
    chars = list(set(data))
    data_size, vocab_size = len(data), len(chars)
    print("Data has {} characters, {} unique.".format(data_size, vocab_size))

    char_to_ix = { ch:i for i,ch in enumerate(chars) }
    ix_to_char = { i:ch for i,ch in enumerate(chars) }

    print("Turn an array of characters to an array of numbers:")
    data_ix = [char_to_ix[char] for char in data]
    print("  data[0]={} has been turned to: \n  data_ix[0]={}".format(data[0], data_ix[0]))

    data_one_hot = np.zeros(shape=(data_size, vocab_size), dtype=float)
    for i in range(len(data_ix)):
        idx = data_ix[i]
        data_one_hot[i,idx] = 1.0
    print("Turn an array of numbers to an array of one-hot encoded vectors:")
    print("  data_ix[0]={} has been turned to: \n  data_one_hot[0]={}".format(data_ix[0], data_one_hot[0]))
    print("Returning data_one_hot, data_size, vocab_size, ix_to_char")
    return data_one_hot, data_size, vocab_size, ix_to_char

In [42]:
def vector_to_char(vec, ix_to_char):
    """Returns most probable character represented by the 'one-hot' vector of probabilities."""
    return ix_to_char[np.argmax(vec)]

In [43]:
data_one_hot, data_size, vocab_size, ix_to_char = load_data('data/first_names.txt')

Loaded data from data/first_names.txt
Data has 36122 characters, 27 unique.
Turn an array of characters to an array of numbers:
  data[0]=J has been turned to: 
  data_ix[0]=9
Turn an array of numbers to an array of one-hot encoded vectors:
  data_ix[0]=9 has been turned to: 
  data_one_hot[0]=[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Returning data_one_hot, data_size, vocab_size, ix_to_char


In [44]:
vector_to_char(data_one_hot[0], ix_to_char)

'J'